## Data Processing

In [ ]:
from rag_kg.pipeline.process_data import filter_data, create_entity_relation_data

In [ ]:
filtered_data = filter_data(["paris"])
relations = create_entity_relation_data(filtered_data)

## Construct Knowledge Graph

In [3]:
from rag_kg.knowledge_graph.script import clear_database, populate_database, get_all_entities_relations

In [4]:
clear_database()
populate_database()

In [5]:
result = get_all_entities_relations()

## Query Data

In [6]:
from rag_kg.pipeline.infer import get_result

In [7]:
query = "What are some sport events that held and some books written in cities of France?"
print(get_result(query))

INFO: rag_kg.pipeline.infer: Processing Query: What are some sport events that held and some books written in cities of France?
INFO: rag_kg.pipeline.infer: Independent Queries: [[{'index': 0, 'query': 'What are the cities in France?', 'dependencies': [], 'entity2': {'name': 'France', 'type': 'country'}, 'entity1_type': 'city', 'relation': 'located_in'}], [{'index': 1, 'query': 'What are some sport events held in <city>?', 'dependencies': [0], 'entity2': {'name': '<city>', 'type': 'city'}, 'entity1_type': 'event', 'relation': 'held_in'}, {'index': 2, 'query': 'What are some books written in <city>?', 'dependencies': [0], 'entity2': {'name': '<city>', 'type': 'city'}, 'entity1_type': 'book', 'relation': 'written_in'}]]


Here are some sport events held in cities of France: 1931 European Rowing Championships, 1957 World Fencing Championships, 1968 European Karate Championships, 1970 Paris Open Indoor Championships, 1971 European Karate Championships, 1981 Paris Open, 1982 Paris Open, 1982 World Sambo Championships, 1986 Paris Open, 1988 Paris Open, 1992 Open Clarins, 1992 Paris Open, 1995 Paris Open, 1996 Paris Open, 1997 Paris Open, 1999 French Open, 1999 Paris Masters, 2000 Paris Masters, 2005 French Open in badminton, 2008 Paris Masters, 2016 Judo Grand Slam, Art Blakey concert, and German occupation.

There are no books written in cities of France in the retrieved knowledge graph.
